In [8]:
import cv2

# Funkcja przetwarzania obrazu (detekcja ruchu)
def detect_motion(prev_frame, frame):
    # Konwersja do skali szarości
    gray_prev = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
   
    # Obliczenie różnicy klatek
    diff = cv2.absdiff(gray_prev, gray_frame)

    # cv2.imshow('Frame with Motion Detection', diff)
   
    # Progowanie różnicy
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # cv2.imshow('Frame with Motion Detection', thresh)
   
    # Zastosowanie morfologicznej operacji otwarcia, aby usunąć szum
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
    # kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (20,20))
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    # opened = cv2.dilate(opened, kernel2)
    # opened = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel2)

    cv2.imshow('Frame with Motion Detection', opened)
   
    # Znajdź kontury ruchu
    contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
   
    # Narysuj prostokąt wokół obszarów z ruchem
    for contour in contours:
        if cv2.contourArea(contour) > 100:  # minimalna powierzchnia obszaru z ruchem
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
   
    return frame

# Wczytaj wideo
cap = cv2.VideoCapture('filmy/7.mp4')  # Zmień 'nazwa_pliku.mp4' na ścieżkę do swojego pliku wideo

# Wczytaj pierwszą klatkę
ret, prev_frame = cap.read()

while(cap.isOpened()):
    ret, frame = cap.read()  # Odczytaj ramkę z wideoqq

    # Sprawdź czy ramka została poprawnie odczytana
    if ret:
        # Detekcja ruchu
        frame_with_motion = detect_motion(prev_frame, frame)

        # Wyświetl ramkę z detekcją ruchu
        cv2.imshow('Frame with Motion Detection', frame_with_motion)

        # Zaktualizuj poprzednią ramkę
        prev_frame = frame.copy()

        # Sprawdź czy użytkownik nacisnął klawisz 'q', jeśli tak, przerwij pętlę
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

# Zwolnij zasoby i zamknij okna
cap.release()
cv2.destroyAllWindows()
    